# Imports

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model, layers
from tensorflow import keras

In [ ]:
from random_point_generator import random_point_cloud
from keras.layers import Lambda, merge

# Constants

In [ ]:
DENSITY_1 = 1000
DENSITY_2 = 1000
MU_1 = 2.71e-3
MU_2 = 1.355e-3
SIGMA = 5.42e-2
T_SIM = 0
T_STEP = 1e-4
DIV_N_WALLS = 2
SCALING = 5e-7
LAYER_THICKNESS = 3
N_STEPS = int(T_SIM/T_STEP) + 1
POINTS_INT = 1 #1000
POINTS_INL = 50
POINTS_WALLS = 250
VEL_1D = -0.1875
VEL_2D = 0.1875
VEL_C = 1.5
LEN = 1e-5
EPS = 1e-6

# Input preprocessing

In [ ]:
path = "path/to/your/folder"

In [ ]:
inlet_1d = (np.loadtxt(path + "pinn-inlet-1d.asc", skiprows=0).astype(np.float32))*SCALING
inlet_2d = (np.loadtxt(path + "pinn-inlet-2d.asc", skiprows=0).astype(np.float32))*SCALING
inlet_c = (np.loadtxt(path + "pinn-inlet-c.asc", skiprows=0).astype(np.float32))*SCALING
interior = (np.loadtxt(path + "pinn-interior.asc", skiprows=0).astype(np.float32))*SCALING
outlet = (np.loadtxt(path + "pinn-outlet.asc", skiprows=0).astype(np.float32))*SCALING
walls  = (np.loadtxt(path + "pinn-walls.asc", skiprows=0).astype(np.float32))*SCALING

In [ ]:
inlet_1d = np.delete(inlet_1d,2,1)
inlet_2d = np.delete(inlet_2d,2,1)
inlet_c = np.delete(inlet_c,2,1)
interior = np.delete(interior,2,1)
outlet = np.delete(outlet,2,1)
walls = np.delete(walls,2,1)

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(inlet_1d[:,0], inlet_1d[:,1],c='red')
plt.scatter(inlet_2d[:,0], inlet_2d[:,1],c='red')
plt.scatter(inlet_c[:,0], inlet_c[:,1],c='blue')
plt.scatter(interior[::1000,0], interior[::1000,1],c='green')
plt.scatter(outlet[:,0], outlet[:,1],c='yellow')
plt.scatter(walls[:,0], walls[:,1],c='brown')
plt.ylim([-5e-5,5e-5])
plt.xlim([0,2e-4])
plt.yticks(-5e-6*np.arange(-10,11,1))
plt.show()

In [ ]:
#u_inlet_d1 = np.zeros(inlet_1d[:,0].shape)
v_inlet_d1 = (VEL_1D*(1 - ((inlet_1d[:,0] - np.average(inlet_1d[:,0]))/(LEN/2))**2)).astype(np.float32)
#a_inlet_d1 = np.zeros(inlet_1d[:,0].shape)

#u_inlet_d2 = np.zeros(inlet_2d[:,0].shape)
v_inlet_d2 = (VEL_2D*(1 - ((inlet_2d[:,0] - np.average(inlet_2d[:,0]))/(LEN/2))**2)).astype(np.float32)
#a_inlet_d2 = np.zeros(inlet_2d[:,0].shape)


u_inlet_c = VEL_C*(1 - ((inlet_c[:,1])/(LEN/2))**2).astype(np.float32)
#v_inlet_c = np.zeros(inlet_c[:,1].shape)
a_inlet_c = np.ones(inlet_c[:,1].shape).astype(np.float32)

#p_outlet = np.zeros(outlet[:,0].shape)

#u_walls = np.zeros(walls[:,0].shape)
#v_walls = np.zeros(walls[:,1].shape)

# Normalizing the domain
#### length of diagonal=sqrt((maxX-minX)*(maxX-minX) + (maxY-minY)*(maxY-minY))   

In [ ]:
def mag(location: np.ndarray) -> np.float32:
    return np.linalg.norm(location.max(axis=0) - location.min(axis=0))

In [ ]:
norm_inlet_1d = mag(inlet_1d)
norm_inlet_2d = mag(inlet_2d)
norm_inlet_c = mag(inlet_c)

### NN Model

#### 1. Create three parallel short NNs.

#### 2. Each layer begins with a Lambda layer, which is essentially a fixed layer with no trainable parameters. These lambda layers will remap the co-ordinates from the original space to (0,1) and send it to the NN. 

#### 3. Concate the Layers and then add few layers for allow it to learn to use specific NN architecture before the concate 

In [ ]:
inputs = layers.Input(2)

d1 = Lambda(lambda x:(x - tf.convert_to_tensor(inlet_1d.min(axis=0)))/tf.convert_to_tensor(norm_inlet_1d))(inputs)
d1 = layers.Dense(LAYER_THICKNESS,name="d1_dense_1")(d1)
d1 = tf.nn.swish(d1)
d1 = layers.Dense(LAYER_THICKNESS,name="d1_dense_2")(d1)
d1 = tf.nn.swish(d1)

d2 = Lambda(lambda x:(x - tf.convert_to_tensor(inlet_2d.min(axis=0)))/tf.convert_to_tensor(norm_inlet_2d))(inputs)
d2 = layers.Dense(LAYER_THICKNESS,name="d2_dense_1")(d2)
d2 = tf.nn.swish(d2)
d2 = layers.Dense(LAYER_THICKNESS,name="d2_dense_2")(d2)
d2 = tf.nn.swish(d2)

c = Lambda(lambda x:(x - tf.convert_to_tensor(inlet_c.min(axis=0)))/tf.convert_to_tensor(norm_inlet_c))(inputs)
c = layers.Dense(LAYER_THICKNESS,name="c_dense_1")(c)
c = tf.nn.swish(c)
c = layers.Dense(LAYER_THICKNESS,name="c_dense_2")(c)
c = tf.nn.swish(c)

model_concate = layers.Concatenate()((d1,d2,c))
x = tf.nn.swish(model_concate)
x = layers.Dense(LAYER_THICKNESS)(x)
x = tf.nn.swish(x)
outputs = layers.Dense(2)(x)

In [ ]:
model = Model(inputs=inputs,outputs=outputs)
model.compile(loss= "mse", optimizer = keras.optimizers.Adam(learning_rate=1e-3))

In [ ]:
model.summary()

In [ ]:
X = np.vstack((inlet_1d,inlet_2d,inlet_c))
v = np.hstack((v_inlet_d1,v_inlet_d2,np.zeros(u_inlet_c.size)))
u = np.hstack((np.zeros(v_inlet_d1.size),np.zeros(v_inlet_d2.size),u_inlet_c))
y = np.vstack((u,v)).T

In [ ]:
his = model.fit(X,y,verbose=0,epochs=2000)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(his.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.yscale('log')
plt.legend(['loss'], loc='upper left')
plt.show()

# Comparing the predictions to the response

In [ ]:
X_inlet_d1 = random_point_cloud(inlet_1d,0,POINTS_INL)

true_v_inlet_d1 = random_point_cloud(v_inlet_d1,0,POINTS_INL)

pred = model.predict(X_inlet_d1)

In [ ]:
plt.scatter(X_inlet_d1[:,0],pred[:,1])
plt.scatter(random_point_cloud(inlet_1d,0,POINTS_INL)[:,0],true_v_inlet_d1);

In [ ]:
X_inlet_d2 = random_point_cloud(inlet_2d,0,POINTS_INL)

true_v_inlet_d2 = random_point_cloud(v_inlet_d2,0,POINTS_INL)

pred = model.predict(X_inlet_d2)

In [ ]:
plt.scatter(X_inlet_d2[:,0],pred[:,1])
plt.scatter(random_point_cloud(inlet_2d,0,POINTS_INL)[:,0],true_v_inlet_d2);

In [ ]:
X_inlet_c = random_point_cloud(inlet_c,0,POINTS_INL)

true_u_inlet_c = random_point_cloud(u_inlet_c,0,POINTS_INL)

pred = model.predict(X_inlet_c)

In [ ]:
plt.scatter(X_inlet_c[:,1],pred[:,0])
plt.scatter(random_point_cloud(inlet_c,0,POINTS_INL)[:,1],true_u_inlet_c);